In [1]:
import numpy as np
import pandas as pd
from scipy.stats import laplace
import scipy.stats
import psycopg2
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import math

In [2]:
def get_index(date_idx, n, degree):
    """Calculates the path of index in full binary string

    Parameters:
    date_idx (int): The node in the bouttom layer we want to calculate a path to. 
    The bottom layer has index from 0 to 2**h-1
    n_layers (int): The height of the full binary tree. 

    Returns:
    list: of index in the path from the starting from the bottom and going up

    """
    idx = []
    for i in np.arange(0,n):
        if i == 0:
            idx.append(int(date_idx))
        else:
            idx.append(int(idx[i-1]//degree))
    idx.append(0)
    return idx

In [3]:
print(get_index(0,1,3))
print(get_index(0,2,3))
print(get_index(0,3,3))
print(get_index(12,3,3))
print(get_index(15,3,3))
print(get_index(9,3,3))

[0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[12, 4, 1, 0]
[15, 5, 1, 0]
[9, 3, 1, 0]


In [4]:
print(get_index(12,3,4))


[12, 3, 0, 0]


In [5]:
def get_group(degree, idx, level):
    """Calculates the path of index in full binary string

    Parameters:
    date_idx (int): The node in the bouttom layer we want to calculate a path to. 
    The bottom layer has index from 0 to 2**h-1
    n_layers (int): The height of the full binary tree. 0 index

    Returns:
    list: of index in the path from the starting from the bottom and going up

    """
    if level == 0:
        return id
    elif idx == 0:
        return np.arange(0,degree)
    else:
        group_index = idx //degree
        
        #Array of index at level
        level_indicis = np.arange(0,degree**level)
        split_ratio = (len(level_indicis) // degree)
        
        level_indicis_split = np.array_split(level_indicis, split_ratio)
        #print(level)

        #print(group_index)
        #print(level_indicis)
        #print(split_ratio)
        #print(level_indicis_split)
            
        return level_indicis_split[group_index]

In [6]:
def turns_right(path, degree):
    #print(path)
    #0 is left 1 is right
    direction_lst = []
    for i in range(len(path)-1):
        #print(f'i = {i}')
        current = path[i]
        nxt = path[i+1]

        if nxt == 0:
            #We went left
            direction_lst.append(0)
            
        elif nxt == current*degree + degree - 1:
            #We went right
            direction_lst.append(1)
            
        else: 
            direction_lst.append(0)
            
    return direction_lst

In [7]:
path = [0, 0, 1, 2, 5, 10]
print(turns_right(path,2))
path = [0, 1, 3, 10]
print(turns_right(path,3))

[0, 1, 0, 1, 0]
[0, 0, 0]


In [8]:
def turns_left(path, degree):

    #0 is left 1 is right
    direction_lst = []
    for i in range(len(path)-1):
        #print(f'i = {i}')
        current = path[i]
        nxt = path[i+1]

        #Checks
        if nxt == 0:
            #We went left
            direction_lst.append(1)
        #Checks
        elif current == 0 and current < nxt:
            #We went right
            direction_lst.append(0)
        elif nxt == degree * current:
            #We went left
            direction_lst.append(1)
        else:
            #We went right
            direction_lst.append(0)
            
    return direction_lst

In [9]:
path = [0,1,3,11]
turns_left(path,3)

[0, 1, 0]

In [10]:
def left_or_right(current, nxt):
    #0 is left 1 is right
    #print(f'current = {current}, nxt = {nxt}')
    if nxt == 0:
        #print('nxt == 0')
        return 0
    if current == 0 and current < nxt:
        #print('current == 0 and current < nxt')
        return 1
    elif 2* current < nxt:
        #print('2* current < nxt')
        return 1
    else:
        #print('else')
        return 0
    
def left_or_right_list(path):
    #print(path)
    #0 is left 1 is right
    direction_lst = []
    for i in range(len(path)-1):
        #print(f'i = {i}')
        current = path[i]
        nxt = path[i+1]
        print(f'current = {current}, nxt = {nxt}')
        if nxt == 0:
            #print('nxt == 0')
            direction_lst.append(0)
        elif current == 0 and current < nxt:
            #print('current == 0 and current < nxt')
            direction_lst.append(1)
        elif 2* current < nxt:
            #print('2* current < nxt')
            direction_lst.append(1)
        else:
            #print('else')            
            direction_lst.append(0)
            
    return direction_lst
#p = [0, 1, 5, 23]
#p = [0, 1, 4, 19]
#p = [0, 3, 13, 55]
#print(turns_left(p, degree))
"""
Paths
[ 0  1  5 23]
current = 0, nxt = 1
current == 0 and current < nxt
current = 1, nxt = 5
current * degree != nxt
current = 5, nxt = 23
current * degree != nxt
Turns
[1, 1, 1]

"""

'\nPaths\n[ 0  1  5 23]\ncurrent = 0, nxt = 1\ncurrent == 0 and current < nxt\ncurrent = 1, nxt = 5\ncurrent * degree != nxt\ncurrent = 5, nxt = 23\ncurrent * degree != nxt\nTurns\n[1, 1, 1]\n\n'

In [11]:
degree = 4
n_layers = 3
print(get_index(0, n_layers, degree))
print(get_index(32, n_layers, degree))


[0, 0, 0, 0]
[32, 8, 2, 0]


In [17]:
"""

hhs = [[3241], [833, 843, 812, 823], [181, 200, 199, 219, 170, 217, 205,
        202, 221, 184, 213, 182, 211, 210, 173, 208],[56, 47, 60, 50, 56,
        62, 62, 51, 61, 47, 52, 41, 54, 63, 45, 46, 38, 46, 52, 53, 59, 58,
        48, 46, 41, 48, 33, 48, 58, 44, 38, 49, 46, 60, 37, 65, 59, 49, 70,
        56, 48, 50, 53, 53, 37, 46, 62, 55, 42, 50, 40, 49, 44, 52, 55, 59,
        49, 61, 43, 49, 51, 63, 55, 51]]
hhs = [np.array(a) for a in hhs]
print(hhs)
hhs = [[1175], [420, 405, 412,], [123, 133, 116, 139, 136, 144, 146, 125, 113,],
[62, 44, 30, 39, 44, 41, 46, 54, 32, 41, 35, 45, 54, 48, 44, 43, 44, 41,
 41, 45, 41, 42, 38, 53, 39, 49, 49,]]
hhs = [np.array(a) for a in hhs]
print(hhs)
"""
"""

hhs = [np.arange(0,1),
       np.arange(0,2),
       np.arange(0,4),
       np.arange(0,8),
       np.arange(0,16),
       np.arange(0,32)]

       
hhs = [np.arange(0,1),
       np.arange(0,3),
       np.arange(0,9),
       np.arange(0,27)]
       
       
hhs = [np.arange(0,1),
       np.arange(0,4),
       np.arange(0,8),
       np.arange(0,16),
       np.arange(0,64)]


"""
hhs = [np.arange(0,1),
       np.arange(0,2),
       np.arange(0,4),
       np.arange(0,8),
       np.arange(0,16),
       np.arange(0,32)]
hhs = [np.array(a) for a in hhs]
print(hhs)

[array([0]), array([0, 1]), array([0, 1, 2, 3]), array([0, 1, 2, 3, 4, 5, 6, 7]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])]


# When we start from 0

In [13]:
degree = 3
n_layers = 3 #self.h

idx_0 = 0
idx_1 = 22 

idx_left = idx_0-1
idx_right = idx_1+1

#path_to_left = np.flip(np.array(get_index(idx_left,n_layers, degree)))
path_to_right = np.flip(np.array(get_index(idx_right,n_layers, degree)))
print('Paths')
#print(path_to_left)
print(path_to_right)

turns_left_lst = turns_left(path_to_right, degree)
#left_or_right_list_rightside = turns_right(path_to_right, degree)
print('Left and right list')
#print(left_or_right_list_leftside)
print(turns_left_lst)

if idx_0 == 0:
    #level_offset = len(path_to_left)-len(left_or_right_list((np.array(path_to_left)[path_to_right != path_to_left])))
    level_offset = 1
    print(f'level_offset = {level_offset}')
   
    for i in range(len(turns_left_lst)):
        print(f'i = {i}')
        
        if turns_left_lst[i] == 0:
            group = get_group(degree, path_to_right[i+level_offset], i+level_offset)
            idx_sss = np.where(group == path_to_right[i+level_offset])[0][0]
            #print(f'group = {group}')
            #print(f'idx_sss = {idx_sss}')
            #print(f'i+level_offset = {i+level_offset}')
            #print(f'group[:idx_sss] = {group[:idx_sss]}')
            print(f'hhs[i+level_offset] {hhs[i+level_offset]}')
            count_nodes = hhs[i+level_offset][group[:idx_sss]]
            print('We need to count the following nodes')
            for node in count_nodes:
                print(f'node = {node}')
        else:
            print('nothing to count')
for level in hhs:
    print(level)

Paths
[ 0  2  7 23]
Left and right list
[0, 0, 0]
level_offset = 1
i = 0
hhs[i+level_offset] [0 1 2]
We need to count the following nodes
node = 0
node = 1
i = 1
hhs[i+level_offset] [0 1 2 3 4 5 6 7 8]
We need to count the following nodes
node = 6
i = 2
hhs[i+level_offset] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
We need to count the following nodes
node = 21
node = 22
[0]
[0 1 2]
[0 1 2 3 4 5 6 7 8]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


# When we start from max

In [14]:
degree = 2
n_layers = 5

idx_0 = 11
idx_1 = 31 

idx_left = idx_0-1
idx_right = idx_1+1

path_to_left = np.flip(np.array(get_index(idx_left,n_layers, degree)))
#path_to_right = np.flip(np.array(get_index(idx_right,n_layers, degree)))
print('Paths')
print(path_to_left)
#print(path_to_right)

#turns_left_lst = turns_left(path_to_right, degree)
turns_right_lst = turns_right(path_to_left, degree)
#left_or_right_list_rightside = turns_right(path_to_right, degree)
print('Left and right list')
#print(left_or_right_list_leftside)
print(turns_right_lst)

if True:
    level_offset = 1
   
    for i in range(len(turns_right_lst)):
        print(f'i = {i}')
        
        if turns_right_lst[i] == 0:
            
            group = get_group(degree, path_to_left[i+level_offset], i+level_offset)
            idx_sss = np.where(group == path_to_left[i+level_offset])[0][0]
            print(f'hhs[i+level_offset] {hhs[i+level_offset]}')
            count_nodes = hhs[i+level_offset][group[idx_sss+1:]]
            print('We need to count the following nodes')
            for node in count_nodes:
                print(f'node = {node}')
        else:
            print('nothing to count')
        
for level in hhs:
    print(level)

Paths
[ 0  0  1  2  5 10]
Left and right list
[0, 1, 0, 1, 0]
i = 0
hhs[i+level_offset] [0 1 2]
We need to count the following nodes
node = 1
i = 1
nothing to count
i = 2
hhs[i+level_offset] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
We need to count the following nodes
node = 3
i = 3
nothing to count
i = 4


IndexError: list index out of range

# Random search



In [24]:
degree = 2
n_layers = 5

#32 #48 #24 FOR DEGREE FOUR
idx_0 = 25

idx_1 = 30

#print(get_index(idx_0, n_layers, degree))
#print(get_index(idx_1, n_layers, degree))

idx_left = idx_0-1
idx_right = idx_1+1

path_to_left = np.flip(np.array(get_index(idx_left,n_layers, degree)))
path_to_right = np.flip(np.array(get_index(idx_right,n_layers, degree)))
print('Paths')
print(path_to_left)
print(path_to_right)
#For max
    
turns_left_lst = turns_left(path_to_right, degree)
turns_right_lst = turns_right(path_to_left, degree)
print('Left and right list')
print(turns_left_lst)
print(turns_right_lst)


level_offset = 1

left_count = []
left_count_group = []
for i in range(len(turns_left_lst)):
    print(f'i = {i}')
        
    if turns_left_lst[i] == 0:
        group = get_group(degree, path_to_right[i+level_offset], i+level_offset)
        idx_sss = np.where(group == path_to_right[i+level_offset])[0][0]
        
        left_count_group.append(group[:idx_sss])

        count_nodes = hhs[i+level_offset][group[:idx_sss]]
        left_count.append(count_nodes)
    else:
        left_count_group.append(np.array([]))
        left_count.append(np.array([]))
    
#The search right side
right_count = []
right_count_group = []
for i in range(len(turns_right_lst)):
        
    if turns_right_lst[i] == 0:
            
        group = get_group(degree, path_to_left[i+level_offset], i+level_offset)
        idx_sss = np.where(group == path_to_left[i+level_offset])[0][0]
        
        right_count_group.append(group[idx_sss+1:]) 
        
        count_nodes = hhs[i+level_offset][group[idx_sss+1:]]
        
        right_count.append(count_nodes)

    else:
        right_count_group.append(np.array([]))
        right_count.append(np.array([]))
print('Counting nodes')

print(left_count)
print(left_count_group)

print(right_count)
print(right_count_group)
#print(left_count)
#print(right_count)
for i in range(len(left_count_group)):
    print(f'At level {level_offset + i}')
    """
    print('Left size')
    print(left_count_group[i].size)
    print('right size')
    print(right_count_group[i].size)
    """
    if left_count_group[i].size != 0 and right_count_group[i].size != 0:
        print('Both not zero')
        group_left = get_group(degree, left_count_group[i][0], i+ level_offset)
        group_right = get_group(degree, right_count_group[i][0], i+ level_offset)

        if not (np.array_equal(group_left,group_right)):
            for node in left_count_group[i]:
                print(f'hhs[i+level_offset] 1 {hhs[i+level_offset]}')
                print(f'node = {node}')
            for node in right_count_group[i]:
                print(f'hhs[i+level_offset] 2 {hhs[i+level_offset]}')
                print(f'node = {node}')
        else:
            #print(left_count_group[i])
            #print(right_count_group[i])
            #print(np.intersect1d(left_count_group[i], right_count_group[i]))
            count_nodes = np.intersect1d(left_count_group[i], right_count_group[i])
            for node in count_nodes:
                print(f'hhs[i+level_offset] 3 {hhs[i+level_offset]}')
                print(f'node = {node}')
                #range_count = range_count + self.OLH_aggre(node, np.sum(self.tree_levels[i+level_offset]), len(self.tree_levels[i+level_offset]))
    
    print('Here we count')        
    if left_count_group[i].size != 0 and right_count_group[i].size == 0:
        if path_to_left[i] != path_to_right[i]:
            print('Left not zero')
            for node in left_count_group[i]:
                print(f'hhs[i+level_offset] 4 {hhs[i+level_offset]}')
                print(f'node = {node}')
    
    elif right_count_group[i].size != 0 and left_count_group[i].size == 0:
        if path_to_left[i] != path_to_right[i]:
            print('Right not zero')
            for node in right_count_group[i]:
                print(f'hhs[i+level_offset] 5 {hhs[i+level_offset]}')
                print(f'node = {node}')
"""
"""
        
for level in hhs:
    print(level)

Paths
[ 0  1  3  6 12 24]
[ 0  1  3  7 15 31]
Left and right list
[0, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
i = 0
i = 1
i = 2
i = 3
i = 4
Counting nodes
[array([0]), array([2]), array([6]), array([14]), array([30])]
[array([0]), array([2]), array([6]), array([14]), array([30])]
[array([], dtype=float64), array([], dtype=float64), array([7]), array([13]), array([25])]
[array([], dtype=float64), array([], dtype=float64), array([7]), array([13]), array([25])]
At level 1
Here we count
At level 2
Here we count
At level 3
Both not zero
Here we count
At level 4
Both not zero
hhs[i+level_offset] 1 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
node = 14
hhs[i+level_offset] 2 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
node = 13
Here we count
At level 5
Both not zero
hhs[i+level_offset] 1 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
node = 30
hhs[i+level_offset] 2 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25

In [ ]:
"""
print(get_group(0,1,3,0))
print(get_group(0,2,3,0))
print(get_group(0,3,3,0))
print(get_group(12,3,3,0))
print(get_group(15,3,3,0))
"""
degree = 3
n_layers = 3
#print(get_group(0,n_layers, degree, 2))
#print(get_group(9,3,3,3))
#print(get_group(12,3,3,3))
#print(get_group(21,3,3,3))
print(get_group(3, n_layers, degree, 2))
print(get_group(21, n_layers, degree, 3))
print(get_group(9, n_layers, degree, 4))
#print(get_group(3,3,3,2))
#print(get_group(10,3,3,3))

In [ ]:
degree = 4
n_layers = 3
print(get_group(4, n_layers, degree, 2))
print(get_group(24, n_layers, degree, 3))
degree = 2
n_layers = 3
print(get_group(1, n_layers, degree, 2))
print(get_group(2, n_layers, degree, 3))

In [ ]:
for i in range(0,12):
    print('New')
    print(2**i)
    print(3**i)
    print(4**i)    

In [ ]:
from scipy.special import comb
k = np.array([3, 4])
n = np.array([10, 10])
comb(n, k, exact=False)

print(comb(10, 3, exact=True))

print(comb(10, 3, exact=True, repetition=True))

In [ ]:
def OLH_var(p, N):
    print(f'4*p*(1-p) = {4*p*(1-p)}')
    print(f'N*(2*p-1)**2 = {N*(2*p-1)**2}')
    return 4*p*(1-p)/(N*(2*p-1)**2)
def OLH_var2(p, N):
    print(f'4*p-4*p**2 = {4*p-4*p**2}')
    print(f'(N*2*p-N)**2 = {(N*2*p-N)**2}')
    return (4*p-4*p**2)/(N*2*p-N)**2
def OLH_var3(e, N):
    print(f'4*np.exp(e)= {4*np.exp(e)}')
    print(f'N*(np.exp(e)-1)**2 = {N*(np.exp(e)-1)**2}')
    return (4*np.exp(e))/N*(np.exp(e)-1)**2

In [ ]:
epsilon = 0.2
g = 32
N = np.sum(all_counts[:33])
p = np.exp(epsilon)/(np.exp(epsilon)+g-1)
print(p)
print(OLH_var(p,N))
print(OLH_var2(p,N))
print(OLH_var3(epsilon,N))

In [ ]:
%run psql_functions.py
%run miss_data.py

In [ ]:
query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)
"""
print(result)
print(type(result))
print(type(result[0]))
print((str(result[0][0])))
"""
counts = [(count[0]) for count in result]

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)

In [ ]:
from math import log
"""
[0110000,1101011]=
[0110000,0110000]∪
[0110001,1000000]∪
[1000001,1100000]∪
[1100001,1101000]∪
[1101001,1101010]∪
[1101011,1101011]
"""
print(f"[{int('0110000',2)},{int('1101011',2)}]")
print(f"[{int('0110000',2)},{int('0110000',2)}]")
print(f"[{int('0110001',2)},{int('1000000',2)}]")
print(f"[{int('1000001',2)},{int('1100000',2)}]")
print(f"[{int('1100001',2)},{int('1101000',2)}]")
print(f"[{int('1101001',2)},{int('1101010',2)}]")
print(f"[{int('1101011',2)},{int('1101011',2)}]")
"""
D=32,B=2, the interval[2,22]
can be decomposed into sub-intervals
[2,3]∪[4,7]∪[8,15]∪[16,19]∪[20,21]∪[22,22].
print('[2,3]')
print(get_index(2,5))
print(get_index(3,5))
print('[4,7]')
print(get_index(4,5))
print(get_index(7,5))
print('whole [4,7]')
print(get_index(4,5))
print(get_index(5,5))
print(get_index(6,5))
print(get_index(7,5))
print('[8,15]')
print(get_index(8,5))
print(get_index(15,5))
print('whole [8,15]')
print(get_index(8,5))
print(get_index(9,5))
print(get_index(10,5))
print(get_index(11,5))
print(get_index(12,5))
print(get_index(13,5))
print(get_index(14,5))
print(get_index(15,5))
"""


"""We can construct ititeratively by,
for i decreasing from blog(t−1)cto 0, choosingthe segment of size2ithat is contained in{0, . . . , t−1}and is not contained in a previously chosen segment (if sucha segment exists)
"""
degree = 2
D = 32
#math.log(a,Base)
for i in range(int(np.floor((math.log(D,2)))), 0, -1):
    print(degree**i)
